CNN_CIFAR10 simple CNN with just 3convolution layer for an accuracy of 75%

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# --------------------
# Device
# --------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# --------------------
# Hyperparameters
# --------------------
batch_size = 128
epochs = 50
learning_rate = 0.001

# --------------------
# Data transforms
# --------------------
mean = (0.4914, 0.4822, 0.4465)
std = (0.2470, 0.2435, 0.2616)

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

# --------------------
# Dataset & Loader
# --------------------
trainset = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True, transform=transform_train
)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=True, transform=transform_test
)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

# --------------------
# Simple CNN Model
# --------------------
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc = nn.Linear(128 * 4 * 4, 10)  # 32x32 → 16 → 8 → 4 after pooling
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

model = SimpleCNN().to(device)

# --------------------
# Loss & Optimizer
# --------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# --------------------
# Training function
# --------------------
def train(epoch):
    model.train()
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    acc = 100. * correct / total
    print(f"Epoch [{epoch}] Train Acc: {acc:.2f}%")

# --------------------
# Testing function
# --------------------
def test(epoch):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in testloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    acc = 100. * correct / total
    print(f"Epoch [{epoch}] Test Acc: {acc:.2f}%")
    return acc

# --------------------
# Main loop
# --------------------
best_acc = 0
for epoch in range(1, epochs + 1):
    train(epoch)
    acc = test(epoch)
    best_acc = max(best_acc, acc)

print(f"Best Test Accuracy: {best_acc:.2f}%")


CIFAR-10 with deeper convolution layer and Batch Normalization

Dropout for regularization

Stronger data augmentation

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# --------------------
# Device
# --------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# --------------------
# Hyperparameters
# --------------------
batch_size = 128
epochs = 100
learning_rate = 0.001

# --------------------
# Data transforms (stronger augmentation)
# --------------------
mean = (0.4914, 0.4822, 0.4465)
std = (0.2470, 0.2435, 0.2616)

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

# --------------------
# Deeper CNN Model
# --------------------
class DeepCNN(nn.Module):
    def __init__(self):
        super(DeepCNN, self).__init__()
        # Conv Block 1
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(2,2)
        self.dropout1 = nn.Dropout(0.25)

        # Conv Block 2
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 128, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(128)
        self.pool2 = nn.MaxPool2d(2,2)
        self.dropout2 = nn.Dropout(0.25)

        # Conv Block 3
        self.conv5 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn5 = nn.BatchNorm2d(256)
        self.conv6 = nn.Conv2d(256, 256, 3, padding=1)
        self.bn6 = nn.BatchNorm2d(256)
        self.pool3 = nn.MaxPool2d(2,2)
        self.dropout3 = nn.Dropout(0.25)

        # Fully connected
        self.fc1 = nn.Linear(256*4*4, 512)
        self.bn_fc1 = nn.BatchNorm1d(512)
        self.dropout_fc1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 10)

        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.pool1(x)
        x = self.dropout1(x)

        x = self.relu(self.bn3(self.conv3(x)))
        x = self.relu(self.bn4(self.conv4(x)))
        x = self.pool2(x)
        x = self.dropout2(x)

        x = self.relu(self.bn5(self.conv5(x)))
        x = self.relu(self.bn6(self.conv6(x)))
        x = self.pool3(x)
        x = self.dropout3(x)

        x = x.view(x.size(0), -1)
        x = self.relu(self.bn_fc1(self.fc1(x)))
        x = self.dropout_fc1(x)
        x = self.fc2(x)
        return x

model = DeepCNN().to(device)

# --------------------
# Loss & Optimizer
# --------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# --------------------
# Training function
# --------------------
def train(epoch):
    model.train()
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        # Print every 50 batches
        if batch_idx % 50 == 0:
            print(f"Epoch [{epoch}] Batch [{batch_idx}/{len(trainloader)}] Loss: {loss.item():.3f}")

    acc = 100.*correct/total
    print(f"Epoch [{epoch}] Train Accuracy: {acc:.2f}%")

# --------------------
# Testing function
# --------------------
def test(epoch):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in testloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    acc = 100.*correct/total
    print(f"Epoch [{epoch}] Test Accuracy: {acc:.2f}%")
    return acc

# --------------------
# Main Loop
# --------------------
best_acc = 0.0
for epoch in range(1, epochs+1):
    train(epoch)
    acc = test(epoch)
    best_acc = max(best_acc, acc)

print(f"Best Test Accuracy: {best_acc:.2f}%")


Using device: cpu


100%|██████████| 170M/170M [00:03<00:00, 44.6MB/s]


Epoch [1] Batch [0/391] Loss: 2.469
Epoch [1] Batch [50/391] Loss: 1.649
Epoch [1] Batch [100/391] Loss: 1.578
Epoch [1] Batch [150/391] Loss: 1.502
Epoch [1] Batch [200/391] Loss: 1.331
Epoch [1] Batch [250/391] Loss: 1.179
Epoch [1] Batch [300/391] Loss: 1.142
Epoch [1] Batch [350/391] Loss: 0.934
Epoch [1] Train Accuracy: 50.18%
Epoch [1] Test Accuracy: 64.92%
Epoch [2] Batch [0/391] Loss: 0.841
Epoch [2] Batch [50/391] Loss: 1.000
Epoch [2] Batch [100/391] Loss: 1.122
Epoch [2] Batch [150/391] Loss: 0.974
Epoch [2] Batch [200/391] Loss: 0.829
Epoch [2] Batch [250/391] Loss: 0.941
Epoch [2] Batch [300/391] Loss: 0.849
Epoch [2] Batch [350/391] Loss: 0.928
Epoch [2] Train Accuracy: 67.12%
Epoch [2] Test Accuracy: 66.69%
Epoch [3] Batch [0/391] Loss: 0.828
Epoch [3] Batch [50/391] Loss: 0.737
Epoch [3] Batch [100/391] Loss: 0.698
Epoch [3] Batch [150/391] Loss: 0.724
Epoch [3] Batch [200/391] Loss: 0.661
Epoch [3] Batch [250/391] Loss: 0.884
Epoch [3] Batch [300/391] Loss: 0.755
Epoch